In [5]:
from friendsfamilytest.params import *
from friendsfamilytest.sheethelper import *
from friendsfamilytest.data import *
from friendsfamilytest.utils import *

import pandas as pd 
import seaborn as sns
import sketch
import calplot

from wordcloud import WordCloud
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
data = load_google_sheet()

In [ ]:
data.tail()

In [ ]:
data.duplicated().sum()

In [ ]:
data.shape

In [ ]:
for index, row in data.iterrows():
    print(row['free_text'])
    print(index)

In [2]:
data2 = pd.read_csv('../friendsfamilytest/data/data.csv')

In [3]:
data2.head()

,time,rating,free_text,do_better,surgery,free_text_len,do_better_len,rating_score,classif,classif_scores,sentiment,sentiment_score,improvement_labels,feedback_labels
0,2023-07-13 15:28:22,Extremely likely,"Very friendly caring staff, treated me with re...",NaN,Earls Court Surgery,13.0,3,5.0,NaN,NaN,positive,0.909031,NaN,Overall Patient Satisfaction
1,2023-07-13 15:29:16,Extremely likely,so good so confidential,NaN,Earls Court Surgery,4.0,0,5.0,NaN,NaN,positive,0.925523,NaN,Patient Privacy and Confidentiality
2,2023-07-13 15:29:21,Extremely likely,The doctor was very understanding and very kno...,No. It was the best.,Earls Court Surgery,8.0,5,5.0,NaN,NaN,positive,0.971687,Overall Patient Satisfaction,Medical Staff Competence
3,2023-07-13 15:30:10,Extremely likely,Dr [PERSON] was extremely nice,the receptionist was very unkind,Earls Court Surgery,5.0,5,5.0,NaN,NaN,positive,0.982086,Reception Staff Interaction,Patient-Doctor Communication
4,2023-07-13 15:31:48,Extremely likely,I was spoken to politely on the phone and a te...,No. I was happy with everything.,Earls Court Surgery,44.0,6,5.0,NaN,NaN,neutral,0.755115,Overall Patient Satisfaction,Follow-Up and Continuity of Care


In [16]:
data2.columns
data2['time'] = pd.to_datetime(data2['time'])

In [17]:
data2.drop(columns=['polarity', 'subjectivity', 'neg', 'neu', 'pos', 'compound'], inplace=True)

In [18]:
# Group by sentiment and resample by week, then calculate the mean sentiment_score
weekly_sentiment_means_adjusted = data2.groupby('sentiment').resample('W')['sentiment_score'].mean().unstack(level=0)

# Fill any possible NaN values with 0 (assuming no sentiment means neutral)
weekly_sentiment_means_adjusted = weekly_sentiment_means_adjusted.fillna(0)

weekly_sentiment_means_adjusted.head()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'

In [19]:
import matplotlib
matplotlib.use('TkAgg')  # Use 'Agg', 'TkAgg', 'Qt4Agg', 'Qt5Agg', etc., depending on your preferences and environment
import matplotlib.pyplot as plt

In [20]:
import seaborn as sns

# Melting the DataFrame to long format for easier plotting with seaborn
data_long = weekly_sentiment_means_adjusted.reset_index().melt(id_vars='time', var_name='Sentiment', value_name='Average Score')

# Creating the plot
plt.figure(figsize=(12, 6))
sns.lineplot(data=data_long, x='time', y='Average Score', hue='Sentiment', marker='o')

plt.title('Weekly Sentiment Score Averages', fontsize=16)
plt.xlabel('Week', fontsize=12)
plt.ylabel('Average Sentiment Score', fontsize=12)
plt.legend(title='Sentiment')
plt.tight_layout()
plt.show()

NameError: name 'weekly_sentiment_means_adjusted' is not defined

In [21]:
data2_weekly

NameError: name 'data2_weekly' is not defined

In [ ]:
!pip install --upgrade matplotlib

In [ ]:
def check_column_length(dataframe):
    short_phrases = []

    # Iterate over each entry in the 'do_better' column
    for entry in dataframe['free_text']:
        # Count the number of words in the entry
        word_count = len(str(entry).split())

        # Check if the word count is less than 5
        if word_count < 30:
            short_phrases.append(entry)

    # Print the list of entries with fewer than 5 words
    print("Entries with fewer than 5 words:", short_phrases)

# Example usage
# Assuming 'df' is your DataFrame and it has a column named 'do_better'
# df = pd.DataFrame({'do_better': ['Improve communication', 'Be more timely', 'Feedback', 'Enhance portal usability', 'Listen more']})
# check_do_better_column(df)

In [ ]:
check_column_length(data)

In [ ]:
def check_column_length(dataframe, column_name, word_count_length):
    # Iterate over each entry in the specified column
    for index, entry in enumerate(dataframe[column_name]):
        # Count the number of words in the entry
        word_count = len(str(entry).split())

        # Check if the word count is less than the specified limit
        if word_count < word_count_length:
            # Replace with NaN if the condition is met
            dataframe.at[index, column_name] = pd.nan

    return dataframe

In [ ]:
check_column_length(data2, 'do_better', 4)

In [ ]:
check_column_length(data2, 'free_text', 2)

In [ ]:
data2 = pd.read_csv('../friendsfamilytest/data/data2.csv')

In [ ]:
data2.head(30)

In [ ]:
import numpy as np

In [ ]:
data2.loc[data2['do_better_len'] < 5, 'do_better'] = np.nan
data2.loc[data2['do_better_len'] < 5, 'improvement_labels'] = np.nan

In [ ]:
data2.head(10)

In [ ]:
data2.loc[data2['free_text_len'] < 3, 'free_text'] = np.nan
data2.loc[data2['free_text_len'] < 3, 'feedback_labels'] = np.nan

In [ ]:
data2.head(30)

In [ ]:
data2.shape

In [ ]:
# Saving the indexes corresponding to rows
boolean_mask = (data2['free_text_len']> 10)

# Applying the boolean filteraing
data3 = data2[boolean_mask].reset_index(drop=True)

In [ ]:
data3

In [ ]:
data3.columns

In [ ]:
data3[['time', 'rating', 'free_text', 'do_better', 'surgery', 'free_text_len',
       'do_better_len', 'rating_score', 'classif', 'classif_scores',
       'sentiment', 'sentiment_score', 'improvement_labels', 'polarity',
       'subjectivity', 'feedback_labels']]

In [ ]:
data3.drop(columns=['neg', 'neu', 'pos', 'compound'], inplace=True)

In [ ]:
data3

In [ ]:
import sketch
data3.sketch.ask("WHat is the average Neutral sentiment score?")

In [ ]:
data = pd.read_csv('../friendsfamilytest/data/data.csv')

In [ ]:
data

In [ ]:
data = clean_data(data)

In [ ]:
data.head(30)

In [ ]:
data.to_csv('data.csv', index=False)

In [7]:
calplot.calplot(data2['sentiment_score'], cmap='YlGn', edgecolor='black')

AttributeError: 'RangeIndex' object has no attribute 'year'